# QCBM
Quantum Circuit Born Machine is a model for learning state vectors and using them in machine learning.  
It utilizes an arbitrary state of one qubit and an entanglement of two qubits to learn a state vector.

Install Blueqat

In [ ]:
!pip install blueqat

In this case, I'm going to have them learn a simple pattern.

## Prepare the circuit
Let's build a 4 qubit circuit, using U3 gates and CX.

```
|0> --[input]--U3--*--------X--[repeat]--
|0> --[input]--U3--X--*-----|--[repeat]--
|0> --[input]--U3-----X--*--|--[repeat]--
|0> --[input]--U3--------X--*--[repeat]--[m]--[expt]-[loss]
```

Use SGD for updates and MSE for loss functions.

In [ ]:
from blueqat import Circuit
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import time

np.random.seed(30)

#initial parameters
ainit = [np.random.rand()*np.pi*2 for i in range(36)]

#1qubit state preparation
def arbi(para):
    circ1 = Circuit()
    circ1.u3(para[0],para[1],para[2])[0]
    circ1.u3(para[3],para[4],para[5])[1]
    circ1.u3(para[6],para[7],para[8])[2]
    circ1.u3(para[9],para[10],para[11])[3]
    return circ1
    
#cx loop circuit
def loop():
    circ2 = Circuit()
    circ2.cx[0,1]
    circ2.cx[1,2]
    circ2.cx[2,3]
    circ2.cx[3,0]
    return circ2

#QCBM circuit
def qcbm(a):
    u = Circuit()
    u += arbi(a[0:12])
    u += loop()
    u += arbi(a[12:24])
    u += loop()
    u += arbi(a[24:36])
    u += loop()
    return u

#expectation value
def E(sv):
    return sum(np.abs(sv[:8])**2)-sum(np.abs(sv[8:])**2)

#loss function
def L(p,t):
    return (p-t)**2

def ix(l):
    u = Circuit(4)
    for i in l:
        u.x[i]
    return u

#training data
inp = [[0,1],[1,2],[2,3],[3,0],[0,2],[1,3],[0,1,2],[1,2,3]]
tgt = [1,1,1,1,-1,-1,-1,-1]

#initial parameters
a = ainit.copy()

#result list
ar = []

h = 0.01
e = 0.01

#iterations
nsteps = 3000

start = time.time()
for i in range(nsteps):
    r = np.random.randint(0,len(inp))

    c = ix(inp[r]) + qcbm(a)
    loss = L(E(c.run()),tgt[r])

    ar.append(loss)

    at = [0 for i in range(len(a))]   
    for j in range(len(a)):
        aa = a.copy()
        aa[j] += h
        c = ix(inp[r]) + qcbm(aa)        
        loss2 = L(E(c.run()),tgt[r])
        at[j] = a[j] - e*(loss2 - loss)/h

    a = at

plt.plot(ar)
plt.show()

print(time.time() - start)

In [4]:
#label 1
(ix(inp[0]) + qcbm(a)).m[3].run(shots=1000)

Counter({'0000': 868, '0001': 132})

In [5]:
#label -1
(ix(inp[6]) + qcbm(a)).m[3].run(shots=1000)

Counter({'0000': 158, '0001': 842})